# VASP WAVECAR分析  
首先从[GitHub](https://github.com/QijingZheng/VaspBandUnfolding)Fork了已有的库，目前仅需要其中的`vaspwfc`模块中的`vaspwfc`类。在一开始import所需要的部分即可。  
`from vaspwfc impoirt vaspwfc`  

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import os 

from vaspwfc import vaspwfc

利用Bulk Graphene的WAVECAR生成类。路径Example_Klaus/WAVECAR。  
`vaspwfc(fnm='WAVECAR', lsorbit=False, lgamma=False,gamma_half='x', omp_num_threads=1)`  
- `fnm`:filename of WAVECAR, you could use the relative path representation, e.g. './Example/WAVECAR'  
- 针对Gamma Only的WAVECAR，需要将`lgamma`设置为`True`
```python
# For VASP <= 5.2.x, check
# which FFT VASP uses by the following command:
#
#     $ grep 'use.* FFT for wave' OUTCAR
#
# Then
#
#     # for parallel FFT, VASP <= 5.2.x
#     pswfc = vaspwfc('WAVECAR', lgamma=True, gamma_half='z')
#
#     # for serial FFT, VASP <= 5.2.x
#     pswfc = vaspwfc('WAVECAR', lgamma=True, gamma_half='x')
#
# For VASP >= 5.4, WAVECAR is written with x-direction half grid regardless of
# parallel or serial FFT.
#
#     # "gamma_half" default to "x" for VASP >= 5.4
#     pswfc = vaspwfc('WAVECAR', lgamma=True, gamma_half='x')
```
本次计算中使用的是VASP 5.4.4，因此不需要考虑这个参数的问题。 
- `lsorbit`针对non-collinear计算需要设置为`True`，一般情况下设置为`False`，SOC开启的情况下是否需要设置为True？

In [2]:
filename = "./Example_Klaus/WAVECAR"
wav = vaspwfc(filename)
print("No. K-POINTS: {:4d} \nNo. Bands: {:4d} \nENCUT: {:6.2f}".format(wav._nkpts,wav._nbands,wav._encut))

No. K-POINTS:  120 
No. Bands:   48 
ENCUT: 600.00


`gvectors(ikpt=1, force_Gamma=False, check_consistency=True)`方法可以根据输入的`ikpt`(k点的索引)，生成所有满足(G + k)**2 / 2 < ENCUT的G-vectors。所有的k点坐标可以通过调用`_kvecs`属性查看。首先将所有计算的K-point输出到`KPT`文件中

In [3]:
#写入K points以及相应index信息

f = open(os.path.join(wav._dname,"kpt.txt"),'w')
f.write("ikpt \t K-vectors\n" + "-"*30 + "\n")

for index in range(wav._nkpts):
    msg = str(index+1) + '\t' + "\t".join([str(component) for component in wav._kvecs[index,:]]) + "\n"
    f.write(msg)
f.close()

通过`readBandCoeff(ispin=1, ikpt=1, iband=1, norm=False)`可以提取出需要的态，例如选择`ikpt=1`,`iband=1`,`norm=False`，提取的是Gamma点能量最低态的Cg. 

In [4]:
wav.readBandCoeff(ispin=1, ikpt=1, iband=1, norm=False)
wav.gvectors(ikpt=1,force_Gamma=False)

array([[ 0,  0,  0],
       [ 1,  0,  0],
       [ 2,  0,  0],
       ...,
       [-3, -1, -1],
       [-2, -1, -1],
       [-1, -1, -1]])

In [11]:
def output_gvector_Cg(ikpt=1, iband=1, norm=False, fnm=None,fpath=None):
    """将计算得到的gvectors和Cg按照行输出到文件夹中
    """

    #Filename format: C_{index of band}-{index of kpoint}.txt 
    if not fnm:
        fnm="C_" + str(iband) + "-" +str(ikpt) + ".txt"
    
    if not fpath:
        fpath="."
    f = open(os.path.join(fpath, fnm),'w') 
    header_msg = "# Coefficients of wavefunction for ikpt={:d}\tiband={:d}\n".format(ikpt,iband)
    f.write(header_msg)

    # Generate G vectors and Cg
    Gvectors = wav.gvectors(ikpt,force_Gamma=False)
    Cg = wav.readBandCoeff(ispin=1, ikpt=1, iband=1, norm=False)

    for iG,G in enumerate(Gvectors):
        msg = "\t".join([" ".join([str(Gx) for Gx in G]), str(np.real(Cg[iG])), str(np.imag(Cg[iG])),"\n"])
        f.write(msg)
    
    f.close()
    
output_gvector_Cg(fpath=wav._dname)